In [1]:
# %pip install llama-index-readers-wikipedia

In [3]:
from llama_index.readers.wikipedia import WikipediaReader


In [4]:
reader = WikipediaReader()

In [6]:
cities = ["จังหวัดอยุธยา",
          "จังหวัดปทุมธานี",
          "จังหวัดนครราชสีมา",
          "จังหวัดเชียงใหม่",
          "จังหวัดนครศรีธรรมราช",]

In [5]:
import wikipedia
wikipedia.set_lang('th')

In [7]:
documents = reader.load_data(pages=cities)

In [8]:
documents

[Document(id_='4203', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ จังหวัดพระนครศรีอยุธยาเป็นจังหวัดเดียวในประเทศไทยที่ไม่มีอำเภอเมือง แต่มี อำเภอพระนครศรีอยุธยา เป็นศูนย์กลางการบริหารจัดการด้านต่าง ๆ ชาวบ้านโดยทั่วไปนิยมเรียกอีกชื่อว่า "กรุงเก่า" หรือ "เมืองกรุงเก่า" ห่างจากกรุงเทพมหานครประมาณ 75 กิโลเมตร\n\n\n== ประวัติศาสตร์ ==\n\nพระนครศรีอยุธยาเคยเป็นราชธานี (เมืองหลวง) ของอาณาจักรอยุธยา หรืออาณาจักรสยาม ตลอดระยะเวลา 417 ปี ตั้งแต่ พ.ศ. 1893 กระทั่งเสียกรุงแก่พม่า เมื่อ พ.ศ. 2310 ครั้นเมื่อพระเจ้าตากสินมหาราชทรงสถาปนาราชธานีแห่งใหม่ที่กรุงธนบุรี กรุงศรีอยุธยาก็ไม่ได้กลายเป็นเมืองร้าง ยังมีคนที่รักถิ่นฐานบ้านเดิมอาศัยอยู่และมีราษฎรที่หลบหนีไปอยู่ตามป่ากลับเข้ามาอาศัยอยู่รอบ ๆ เมือง รวมกันเข้าเป็

In [10]:
import faiss

# dimensions of BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatIP(d)

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [13]:
embedding_model_name = '../models/bge-m3'
# embedding_model_name = 'kornwtp/simcse-model-phayathaibert'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cpu')

In [14]:
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
)        
from llama_index.vector_stores.faiss import FaissVectorStore

In [15]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    separator=" ",
)

In [17]:
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
nodes

[TextNode(id_='2080db95-fc50-4dbf-b862-5df5abf5bc17', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4203', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='90b578b743de68c680e945bda9e8935b1df973e46aeea118e519fad36a64c080'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='52f5d2ce-84c9-4ce6-9aa1-0e14ef85c91c', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ce75e1c60f2395dd0ae9af66eb4175e57a5f65e26fdfb48b5b15c014bf44b899')}, text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ จังหวัดพระนครศรีอยุธยาเป็นจังหวัดเดียวในประเทศไทยที่ไม่มีอำเภอเมือง แต่มี อำเภอพระนครศรีอยุธยา เป็นศูนย์กลางการบริหารจัดการด้านต่าง ๆ ชาวบ้านโดยทั่วไปนิยมเรียกอีกชื่อว่า "กรุงเก่า" หรือ "เมืองกรุงเก่า" ห่างจ

In [19]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [20]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=nodes,storage_context=storage_context)

In [21]:
index.storage_context.persist(persist_dir='../index')
